# Scraping data from Youtube

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
weburl = 'https://www.youtube.com'

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get(weburl)

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [5]:
titles = []
channels = []
channel_links = []
views = []
lengths = []
links = []

for a in soup.findAll(attrs={'id':'dismissible','class':'style-scope ytd-rich-grid-media'}):
    title=a.find('yt-formatted-string', attrs={'id':'video-title','class':'style-scope ytd-rich-grid-media'})
    titles.append(title.text)
    channel = a.find('yt-formatted-string', attrs={'id':'text','class':'style-scope ytd-channel-name complex-string'})
    channels.append(channel.text)
    channel_links.append(weburl+channel.a.get('href'))
    view = a.find('div',attrs={'id':'metadata-line','class':'style-scope ytd-video-meta-block'})
    views.append(view.span.text)
    length = a.find('span',attrs={'id':'text','class':'style-scope ytd-thumbnail-overlay-time-status-renderer'})
    if length:
        lengths.append(length.text.strip())
    else:
        lengths.append(None)
    link = a.find('a',attrs={'id':'video-title-link','class':'yt-simple-endpoint focus-on-expand style-scope ytd-rich-grid-media'})
    links.append(weburl+link.get('href'))

In [6]:
youtube_data = pd.DataFrame(data={"Title":titles,"channel name":channels,"channel link":channel_links,"views":views,"Video length":lengths,"Link":links})

In [7]:
youtube_data.shape

(387, 6)

In [8]:
youtube_data.isna().sum()

Title           0
channel name    0
channel link    0
views           0
Video length    7
Link            0
dtype: int64

In [9]:
youtube_data.to_csv("youtube.csv",index=False)